In [1]:
%load_ext autoreload
%autoreload 2

## Shape analysis

The goal is to have a generalized algorithm that for all symmetrically equivalent cluster centers, find nearest atoms from trajectory, and transforms them back to the asymmetric unit. This helps the statistics for performing shape analysis and making plots.

As input:

1. crystal or material structure
    - contains cluster centers
    - includes symmetry operations
2. trajectory
    - typically P1
    - maybe a supercell of crystal structure
    - lattice can be triclinic (non-constrained in simulation)

Algorithm:

1. load clusters (pymatgen structure)
2. load trajectory (pymatgen trajectory)
3. reduce supercell of trajectory to match clusters
    - assert trajectory and cluster lattices match
4. for every unique cluster center, 
    5. for every symmetry operation
    - apply next symmetry operation to cluster center
    - find all trajectory points within X distance
    - copy and map points back to asymmetric unit (reverse symmetry op)
    - subtract cluster center coords
10. perform shape analysis
    - plots, fits, heat maps, msd, etc
   
See: https://github.com/GEMDAT-repos/GEMDAT/pull/166

In [2]:
from __future__ import annotations

from pathlib import Path

import numpy as np

from gemdat import Trajectory
from gemdat.io import load_known_material

trajectory = Trajectory.from_vasprun(
    Path(
        '/home/stef/md-analysis-matlab-example-short/shape_analysis/vasprun.xml'
    ))

supercell = (2, 1, 1)

diff_trajectory = trajectory.filter('Li')

structure = load_known_material('argyrodite')

/home/stef/python/gemdat/.venv/lib64/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [3]:
diff_trajectory.get_lattice(0)

Lattice
    abc : 19.84771 9.923855 9.923855
 angles : 90.0 90.0 90.0
 volume : 1954.6600006747865
      A : 19.84771 0.0 0.0
      B : 0.0 9.923855 0.0
      C : 0.0 0.0 9.923855
    pbc : True True True

In [4]:
# collapse coords
positions = diff_trajectory.positions.reshape(-1, 3)
positions, positions.shape

(array([[0.16145033, 0.33547197, 0.02452775],
        [0.10760483, 0.71007687, 0.50529059],
        [0.37023313, 0.71662174, 0.00237432],
        ...,
        [0.5081269 , 0.69439527, 0.7062491 ],
        [0.91863445, 0.47414316, 0.64892594],
        [0.00493453, 0.24402231, 0.67557806]]),
 (240000, 3))

In [5]:
# collapse supercell
positions = np.mod(positions, 1 / np.array(supercell)) * np.array(supercell)
positions, positions.shape

(array([[0.32290066, 0.33547197, 0.02452775],
        [0.21520966, 0.71007687, 0.50529059],
        [0.74046626, 0.71662174, 0.00237432],
        ...,
        [0.0162538 , 0.69439527, 0.7062491 ],
        [0.8372689 , 0.47414316, 0.64892594],
        [0.00986906, 0.24402231, 0.67557806]]),
 (240000, 3))

In [6]:
from pymatgen import symmetry

sga = symmetry.analyzer.SpacegroupAnalyzer(structure)

symops = sga.get_space_group_operations()
symstruct = sga.get_symmetrized_structure()
lattice = symstruct.lattice
lattice

Lattice
    abc : 9.924 9.924 9.924
 angles : 90.0 90.0 90.0
 volume : 977.3728410239999
      A : 9.924 0.0 6.076697417369166e-16
      B : 1.5959009175390938e-15 9.924 6.076697417369166e-16
      C : 0.0 0.0 9.924
    pbc : True True True

In [7]:
thresh = 1.0  # Å

cluster = []

site0 = symstruct[0]

# TODO: check if symops and symstruct match

for op, site in zip(symops, symstruct):
    # print(op, site)
    # print(site.frac_coords)

    dists = lattice.get_all_distances(site.frac_coords, positions)

    sel = dists < thresh
    close = positions[sel.flatten()]

    print(close.size)

    inversed = op.inverse.operate_multi(close) - site0.frac_coords  # FIXME
    cluster.append(inversed)

13077
16749
8928
10944
11010
10434
9204
9561
9798
14223
12972
10182
16035
12117
7965
12774
4278
9111
14271
13518
14988
12252
14571
11331
14025
16701
11118
13278
10503
13374
19356
11346
9909
12429
16302
5379
14229
17232
11613
11436
5034
15078
18936
16548
16341
21615
7374
10905


In [8]:
cluster = np.vstack(cluster)

In [9]:
cluster.min(axis=0), cluster.max(axis=0)

(array([-1.18298001, -1.18299946, -1.02399203]),
 array([0.81699505, 0.8169879 , 0.97599514]))

## Try reversible symops

In [10]:
tmp = op.operate_multi(positions)
positions2 = op.inverse.operate_multi(tmp)
np.allclose(positions2, positions)

True

## Get distances

In [11]:
dists = symstruct.lattice.get_all_distances(site.frac_coords, positions)

sel = dists < thresh
positions[sel.flatten()].size

10905

## This goes wrong

In [12]:
op.inverse.operate_multi(site.frac_coords)

array([ 0.183, -0.024, -0.183])

In [13]:
site.frac_coords

array([0.317, 0.317, 0.024])

In [14]:
site0.frac_coords

array([0.183, 0.183, 0.024])